<a href="https://colab.research.google.com/github/shiling2007/Python-/blob/main/20_newsgroup_word_embedding_layers_deep_learning_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import files
# files.download('/content/drive/MyDrive/Colab Notebooks/Lease Payment Formula.ipynb') 
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.5g" % x))
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 999)
pd.set_option("max_colwidth", 500)
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#  print('TPU not found')
%load_ext autoreload
%autoreload 2

In [2]:
!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

--2022-10-18 23:07:17--  http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17329808 (17M) [application/x-gzip]
Saving to: ‘news20.tar.gz’

news20.tar.gz       100%[===================>]  16.53M  90.9KB/s    in 78s     

2022-10-18 23:09:03 (216 KB/s) - ‘news20.tar.gz’ saved [17329808/17329808]



In [3]:
!ls -lt

total 16928
drwxr-xr-x 1 root root     4096 Oct 17 13:44 sample_data
-rw-r--r-- 1 root root 17329808 Jan 30  2000 news20.tar.gz


In [ ]:
!tar -xzvf news20.tar.gz

In [5]:
!ls -lt 20_newsgroup

total 532
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 talk.religion.misc
drwxr-xr-x 2 6690 uucp 24576 Apr 20  1998 talk.politics.misc
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 talk.politics.mideast
drwxr-xr-x 2 6690 uucp 24576 Apr 20  1998 talk.politics.guns
drwxr-xr-x 2 6690 uucp 24576 Apr 20  1998 soc.religion.christian
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 sci.space
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 sci.med
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 sci.electronics
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 sci.crypt
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 rec.sport.hockey
drwxr-xr-x 2 6690 uucp 24576 Apr 20  1998 rec.sport.baseball
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 rec.motorcycles
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 rec.autos
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 misc.forsale
drwxr-xr-x 2 6690 uucp 24576 Apr 20  1998 comp.windows.x
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 comp.sys.mac.hardware
drwxr-xr-x 2 6690 uucp 28672 Apr 20  1998 comp.sys.ibm.pc.ha

In [6]:
import os, sys

In [7]:
TEXT_DATA_DIR='20_newsgroup'
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 19997 texts.


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# MAX_NB_WORDS=1000
MAX_SEQUENCE_LENGTH=1000
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT=0.3
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-10-18 23:11:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-10-18 23:11:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-18 23:11:24--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [10]:
!ls -lt

total 3056064
drwxr-xr-x 22 root root       4096 Oct 18 23:10 20_newsgroup
drwxr-xr-x  1 root root       4096 Oct 17 13:44 sample_data
-rw-r--r--  1 root root  862182613 Oct 25  2015 glove.6B.zip
-rw-rw-r--  1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw-rw-r--  1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-rw-r--  1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r--  1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-r--r--  1 root root   17329808 Jan 30  2000 news20.tar.gz


In [11]:
embeddings_index = {}
GLOVE_DIR='/content'

f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [12]:
EMBEDDING_DIM=100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [13]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [14]:
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Input
from tensorflow.keras import Model

In [15]:
len(labels_index)

20

In [17]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=128)

Epoch 1/100
110/110 [==============================] - 4s 31ms/step - loss: 2.4454 - acc: 0.1977 - val_loss: 1.9164 - val_acc: 0.3206
Epoch 2/100
110/110 [==============================] - 3s 28ms/step - loss: 1.6310 - acc: 0.4221 - val_loss: 1.4565 - val_acc: 0.4739
Epoch 3/100
110/110 [==============================] - 3s 28ms/step - loss: 1.2567 - acc: 0.5642 - val_loss: 1.4082 - val_acc: 0.5194
Epoch 4/100
110/110 [==============================] - 3s 29ms/step - loss: 1.0316 - acc: 0.6491 - val_loss: 1.2926 - val_acc: 0.5818
Epoch 5/100
110/110 [==============================] - 3s 30ms/step - loss: 0.8775 - acc: 0.7062 - val_loss: 1.0513 - val_acc: 0.6453
Epoch 6/100
110/110 [==============================] - 3s 30ms/step - loss: 0.7492 - acc: 0.7450 - val_loss: 1.0614 - val_acc: 0.6544
Epoch 7/100
110/110 [==============================] - 3s 29ms/step - loss: 0.6499 - acc: 0.7799 - val_loss: 1.0329 - val_acc: 0.6723
Epoch 8/100
110/110 [==============================] - 3s 29ms

In [18]:

# coding: utf-8

# # 20 Newsgroups text classification with pre-trained word embeddings
# 
# In this script, we'll use pre-trained [GloVe word embeddings]
# (http://nlp.stanford.edu/projects/glove/) for text classification
# using Keras (version $\ge$ 2 is required). This script is largely
# based on the blog post [Using pre-trained word embeddings in a Keras
# model]
# (https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)
# by François Chollet.
# 
# **Note that using a GPU with this script is highly recommended.**
# 
# First, the needed imports. Keras tells us which backend (Theano,
# Tensorflow, CNTK) it will be using.

from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import LSTM, CuDNNLSTM
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing.sequence import pad_sequences

from distutils.version import LooseVersion as LV
from keras import __version__
from keras import backend as K

from sklearn.model_selection import train_test_split

import argparse
import json

import os
import sys

import numpy as np

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))


Using Keras version: 2.9.0 backend: tensorflow


In [19]:
GLOVE_DIR = "/content"

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
    # ## GloVe word embeddings
    # 
    # Let's begin by loading a datafile containing pre-trained word
    # embeddings.  The datafile contains 100-dimensional embeddings for
    # 400,000 English words.


Indexing word vectors.
Found 400000 word vectors.


In [20]:
TEXT_DATA_DIR = "/content/20_newsgroup"

print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))
# ## 20 Newsgroups data set
    # 
    # Next we'll load the [20 Newsgroups]
    # (http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html)
    # data set.
    # 
    # The dataset contains 20000 messages collected from 20 different
    # Usenet newsgroups (1000 messages from each group):
    # 
    # alt.atheism           | soc.religion.christian   | comp.windows.x     | sci.crypt   
    # talk.politics.guns    | comp.sys.ibm.pc.hardware | rec.autos          | sci.electronics
    # talk.politics.mideast | comp.graphics            | rec.motorcycles    | sci.space
    # talk.politics.misc    | comp.os.ms-windows.misc  | rec.sport.baseball | sci.med
    # talk.religion.misc    | comp.sys.mac.hardware    | rec.sport.hockey   | misc.forsale

Processing text dataset
Found 19997 texts.


In [21]:
# Vectorize the text samples into a 2D integer tensor.

MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000 

tokenizer = text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set

VALIDATION_SET, TEST_SET = 1000, 4000

x_train, x_test, y_train, y_test = train_test_split(data, labels, 
                                                    test_size=TEST_SET,
                                                    shuffle=True, random_state=42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size=VALIDATION_SET,
                                                  shuffle=False)

print('Shape of training data tensor:', x_train.shape)
print('Shape of training label tensor:', y_train.shape)
print('Shape of validation data tensor:', x_val.shape)
print('Shape of validation label tensor:', y_val.shape)
print('Shape of test data tensor:', x_test.shape)
print('Shape of test label tensor:', y_test.shape)



Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)
Shape of training data tensor: (14997, 1000)
Shape of training label tensor: (14997, 20)
Shape of validation data tensor: (1000, 1000)
Shape of validation label tensor: (1000, 20)
Shape of test data tensor: (4000, 1000)
Shape of test label tensor: (4000, 20)


In [22]:
# Prepare the embedding matrix:

print('Preparing embedding matrix.')

num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_dim = 100

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Shape of embedding matrix:', embedding_matrix.shape)

Preparing embedding matrix.
Shape of embedding matrix: (10000, 100)


In [26]:
# ### Initialization

print('Build model...')
model = Sequential()

model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
#model.add(Dropout(0.2))

model.add(CuDNNLSTM(128))

model.add(Dense(128, activation='relu'))
model.add(Dense(20, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(model.summary())

# ### Learning

epochs = 100
batch_size=128

json_logging_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: print(json.dumps({
        "epoch": epoch,
        "loss": logs["loss"],

        "val_loss": logs["val_loss"],

    })),
)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    verbose=2, callbacks=[json_logging_callback])

model.save('20ng-rnn.h5')
#         "acc": logs["acc"],
        # "val_acc": logs["val_acc"],

Build model...
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1000, 100)         1000000   
                                                                 
 cu_dnnlstm_3 (CuDNNLSTM)    (None, 128)               117760    
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 20)                2580      
                                                                 
Total params: 1,136,852
Trainable params: 136,852
Non-trainable params: 1,000,000
_________________________________________________________________
None
Epoch 1/100
{"epoch": 0, "loss": 2.723501682281494, "val_loss": 2.439005136489868}
118/118 - 9s - loss: 2.7235 - accuracy: 0.1578 - val_loss: 2.4390 -

In [27]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[28, 7], [28, 14], [46, 4], [15, 14], [21], [15], [43, 4], [2, 28], [43, 14], [26, 1, 7, 38]]
[[28  7  0  0]
 [28 14  0  0]
 [46  4  0  0]
 [15 14  0  0]
 [21  0  0  0]
 [15  0  0  0]
 [43  4  0  0]
 [ 2 28  0  0]
 [43 14  0  0]
 [26  1  7 38]]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              400       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 80.000001


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-10-18 03:09:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-10-18 03:09:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-18 03:09:08--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!ls -lt

total 3039136
drwxr-xr-x 1 root root       4096 Oct 14 19:05 sample_data
-rw-r--r-- 1 root root  862182613 Oct 25  2015 glove.6B.zip
-rw-rw-r-- 1 root root 1037962819 Aug 27  2014 glove.6B.300d.txt
-rw-rw-r-- 1 root root  171350079 Aug  4  2014 glove.6B.50d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200d.txt
-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt


In [ ]:
!pwd

/content


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]
Loaded 400000 word vectors.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            1500      
                                                                 
 flatten_1 (Flatten)         (None, 400)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 401       
                                                                 
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None
Accuracy: 100.000000


In [66]:
from google.colab import files
files.download('news20.tar.gz') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>